<a href="https://colab.research.google.com/github/Brand-Sentiment-Tracking/dev-sentiment-package/blob/main/johnsnow/Train_sentiment_classifier_with_our_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Spark NLP
### Multi-class Sentiment Classification
#### By using SentimentDL

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb)

Only run this block if you are inside Google Colab otherwise skip it

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-04-26 15:16:51--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-04-26 15:16:52--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-04-26 15:16:52--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [2]:
import sparknlp

spark = sparknlp.start(gpu=True)

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version", spark.version)

Spark NLP version:  3.4.3
Apache Spark version 3.0.3


### Load political dataset

In [3]:
# Convert Kaggle data to Pandas dataframe and preprocess
import pandas as pd
from pyspark.sql.functions import rand 

sentiment_url_1 = 'https://raw.githubusercontent.com/Brand-Sentiment-Tracking/python-package/main/data/sentiment_test_data.csv' # Financial Phrase Bank
sentiment_url_2 = 'https://raw.githubusercontent.com/Brand-Sentiment-Tracking/python-package/main/data/SEN_en_R.csv' # SEN data
sentiment_url_3 = 'https://raw.githubusercontent.com/Brand-Sentiment-Tracking/dev-sentiment-package/main/data/labelled_1.csv' # owr extracted data



# # Create a preprocessed spark dataframe
# from pyspark import SparkFiles
# spark.sparkContext.addFile(sentiment_url_3)

# # Read financial dataframe
# df_spark = spark.read.option("header","false").csv("file://"+SparkFiles.get("sentiment_test_data.csv"))
# df_spark = df_spark.withColumnRenamed("_c0", "label").withColumnRenamed("_c1", "text")
# # Shuffle dataset
# df_spark = df_spark.orderBy(rand(2))

# # Read political dataframe
# df_spark = spark.read.option("header","true").csv("file://"+SparkFiles.get("SEN_en_R.csv"))
# df_spark = df_spark.withColumnRenamed("_c0", "idx").withColumnRenamed("headline", "text") \
#                                                  .withColumnRenamed("majority_label", 'label')
# # Replace abbreviations with full lables and filter the unk datapoints (applies only to political dataset)                                         
# df_spark = df_spark.replace("neutr", "neutral").replace("pos", "positive").replace("neg","negative")
# df_spark = df_spark[df_spark.label != "unk"]

# # Read our own dataframe

cols_to_read = ['text',"sentiment (Max's take)"]
df_pandas = pd.read_csv(sentiment_url_3, usecols=cols_to_read)

# Rename sentiment to True_Sentiment
df_pandas.rename(columns={"sentiment (Max's take)":"True_Sentiment"},inplace=True)

# # Make dataset smaller for faster runtime
num_sentences = 500
total_num_sentences = df_pandas.shape[0]
df_pandas.drop(df_pandas.index[num_sentences:total_num_sentences], inplace=True)

# Replace 1, 2 , 3 with negative, neutral, positive
df_pandas["True_Sentiment"].replace({1.0: "negative", 2.0: "neutral", 3.0: "positive"}, inplace=True)

# Downsample majority class neutral
negative_points = df_pandas[df_pandas["True_Sentiment"] == 'negative'].head(50)
neutral_points = df_pandas[df_pandas["True_Sentiment"] == 'neutral'].head(50)
positive_points = df_pandas[df_pandas["True_Sentiment"] == 'positive'].head(50)

print(len(neutral_points))
print(len(positive_points))
print(len(negative_points))

# Downsample negative class

# Concatenate
df_pandas = pd.concat([negative_points, neutral_points, positive_points])

# Shuffle
df_pandas = df_pandas.sample(frac=1)

# # Convert to spark dataframe  
df_spark = spark.createDataFrame(df_pandas)

df_spark.show()


50
50
50
+--------------------+--------------+
|                text|True_Sentiment|
+--------------------+--------------+
|Veterans Minister...|      positive|
|Flipkart says new...|      positive|
|Salt & Straw Brin...|      positive|
|Employer Direct H...|      positive|
|Infinera (NASDAQ:...|      positive|
|Tesla 1Q earnings...|      positive|
|Russia pours in m...|       neutral|
|Kyle Busch steadi...|      positive|
|Several people fe...|      negative|
|‘My body is here,...|       neutral|
|Bank of N.T. Butt...|      negative|
|Huntington Bancsh...|       neutral|
|Report: Transfer ...|      positive|
|New Jersey's recr...|       neutral|
|Woman Convicted i...|      negative|
|Yoshinoya beef bo...|      negative|
|UN court: Colombi...|      negative|
|New technology fo...|       neutral|
|Price Alert: Shar...|      negative|
|Two Children Woun...|      negative|
+--------------------+--------------+
only showing top 20 rows



In [4]:
# Split into train and test
df_spark_train = df_spark.limit(round(df_spark.count()*0.8))
df_spark_test = df_spark.subtract(df_spark_train)

df_spark_train.show()

+--------------------+--------------+
|                text|True_Sentiment|
+--------------------+--------------+
|Veterans Minister...|      positive|
|Flipkart says new...|      positive|
|Salt & Straw Brin...|      positive|
|Employer Direct H...|      positive|
|Infinera (NASDAQ:...|      positive|
|Tesla 1Q earnings...|      positive|
|Russia pours in m...|       neutral|
|Kyle Busch steadi...|      positive|
|Several people fe...|      negative|
|‘My body is here,...|       neutral|
|Bank of N.T. Butt...|      negative|
|Huntington Bancsh...|       neutral|
|Report: Transfer ...|      positive|
|New Jersey's recr...|       neutral|
|Woman Convicted i...|      negative|
|Yoshinoya beef bo...|      negative|
|UN court: Colombi...|      negative|
|New technology fo...|       neutral|
|Price Alert: Shar...|      negative|
|Two Children Woun...|      negative|
+--------------------+--------------+
only showing top 20 rows



The content is inside `text` column and the sentiment is inside `label` column

In [5]:
trainDataset = df_spark_train
trainDataset.show()

+--------------------+--------------+
|                text|True_Sentiment|
+--------------------+--------------+
|Veterans Minister...|      positive|
|Flipkart says new...|      positive|
|Salt & Straw Brin...|      positive|
|Employer Direct H...|      positive|
|Infinera (NASDAQ:...|      positive|
|Tesla 1Q earnings...|      positive|
|Russia pours in m...|       neutral|
|Kyle Busch steadi...|      positive|
|Several people fe...|      negative|
|‘My body is here,...|       neutral|
|Bank of N.T. Butt...|      negative|
|Huntington Bancsh...|       neutral|
|Report: Transfer ...|      positive|
|New Jersey's recr...|       neutral|
|Woman Convicted i...|      negative|
|Yoshinoya beef bo...|      negative|
|UN court: Colombi...|      negative|
|New technology fo...|       neutral|
|Price Alert: Shar...|      negative|
|Two Children Woun...|      negative|
+--------------------+--------------+
only showing top 20 rows



In [6]:
trainDataset.count()

120

In [7]:
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [8]:
# Alternative architecture

document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")
    
bert_cmlm = BertSentenceEmbeddings.pretrained('sent_bert_use_cmlm_en_base', 'en')\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("True_Sentiment")\
      .setMaxEpochs(10)\
      .setEnableOutputLogs(True)\
      .setLr(0.005)\
      .setBatchSize(20)

bert_cmlm_clf_pipeline = Pipeline(
    stages = [
        document,
        bert_cmlm,
        classifierdl
    ])

sent_bert_use_cmlm_en_base download started this may take some time.
Approximate size to download 391.6 MB
[OK!]


In [9]:
# # actual content is inside description column
# document = DocumentAssembler()\
#     .setInputCol("text")\
#     .setOutputCol("document")

# use = UniversalSentenceEncoder.pretrained() \
#  .setInputCols(["document"])\
#  .setOutputCol("sentence_embeddings")


# classsifierdl = ClassifierDLApproach()\
#   .setInputCols(["sentence_embeddings"])\
#   .setOutputCol("class")\
#   .setLabelColumn("True_Sentiment")\
#   .setMaxEpochs(5)\
#   .setEnableOutputLogs(True)

# pipeline = Pipeline(
#     stages = [
#         document,
#         use,
#         classsifierdl
#     ])

In [10]:
# Fit to the training dataset (train the model)
pipelineModel = bert_cmlm_clf_pipeline.fit(trainDataset)

# How to use already trained SentimentDL pipeline or its model

We have two ways of using what we already trained: pipeline or model.

Let's see how we can save the entire pipeline, load it, and do some prediction with that pre-trained pipeline.

## Save and load pre-trained SentimentDL pipeline

In [11]:
# Google Colab is free so it comes with a little memory. 
# It's not possible to save and load in this notebook. But you can do this locally or in a decent machine!

# pipelineModel.save("./sentimentdl_pipeline")
# loadedPipeline = PipelineModel.load("./sentimentdl_pipeline")
# loadedPipeline.transform(YOUR_DATAFRAME)

# Save and load pre-trained SentimentDL model

In [12]:
# hdfs:/ if you are saving it on distributed file systems in Hadoop
pipelineModel.stages[-1].write().overwrite().save('./tmp_sentimentdl_model')



Let's use our pre-trained SentimentDLModel in a pipeline: 

In [13]:
# Alternative architecture

document = DocumentAssembler()\
      .setInputCol("description")\
      .setOutputCol("document")
    
bert_cmlm = BertSentenceEmbeddings.pretrained('sent_bert_use_cmlm_en_base', 'en')\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLModel.load("./tmp_sentimentdl_model") \
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

bert_cmlm_clf_pipeline = Pipeline(
    stages = [
        document,
        bert_cmlm,
        classifierdl
    ])

sent_bert_use_cmlm_en_base download started this may take some time.
Approximate size to download 391.6 MB
[OK!]


In [14]:
# # In a new pipeline you can load it for prediction
# document = DocumentAssembler()\
#     .setInputCol("text")\
#     .setOutputCol("document")

# use = UniversalSentenceEncoder.pretrained() \
#  .setInputCols(["document"])\
#  .setOutputCol("sentence_embeddings")

# classifierdl = ClassifierDLModel.load("./tmp_sentimentdl_model") \
#   .setInputCols(["sentence_embeddings"])\
#   .setOutputCol("class")

# pipeline = Pipeline(
#     stages = [
#         document,
#         use,
#         classifierdl
#     ])


# Evaluation 

Let's evaluatte our SentimentDL model we trained earlier, saved it, and loaded it into a new pipeline by using a test dataset that model has never seen:

In [15]:
testDataset = df_spark_test
preds = pipelineModel.transform(testDataset)

In [16]:
preds.select('True_Sentiment','text',"class.result").show(50, truncate=50)

+--------------+--------------------------------------------------+----------+
|True_Sentiment|                                              text|    result|
+--------------+--------------------------------------------------+----------+
|      negative|Bakery owner accuses Truerlein of ‘broken promi...| [neutral]|
|      positive|Nova Scotia taxi driver leaves $1.68 million to...|[negative]|
|       neutral|UK lawmakers vote on whether to probe Johnson's...| [neutral]|
|      negative|10 Killed, 40 Injured in Bombing at Shiite Mosq...|[negative]|
|      negative|Psychologist: Expect an increase in Georgia’s s...|[positive]|
|       neutral|           CA San Francisco Bay Area Zone Forecast| [neutral]|
|       neutral|CPI figures highlight global economic challenge...| [neutral]|
|       neutral|               Remembering Beavercreek High School| [neutral]|
|      positive|Local favorites Savannah Seafood Shack, Zunzi's...| [neutral]|
|      positive|Transcenta Successfully Passed the A

In [17]:
preds_df = preds.select('True_Sentiment','text',"class.result").toPandas()

In [18]:
# The result is an array since in Spark NLP you can have multiple sentences.
# This means you can add SentenceDetector in the pipeline and feed it into
# UniversalSentenceEncoder and you can have prediction based on each sentence.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [19]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

Let's use `classification_report` from `sklearn` to evaluate the final scores. (keep in mind due to limited resources on a free Google Colab we only used 5 Epochs :)

In [20]:
print (classification_report(preds_df['True_Sentiment'], preds_df['result'] ))

              precision    recall  f1-score   support

    negative       0.80      0.33      0.47        12
     neutral       0.73      0.80      0.76        10
    positive       0.31      0.57      0.40         7

    accuracy                           0.55        29
   macro avg       0.61      0.57      0.54        29
weighted avg       0.66      0.55      0.55        29

